# Crawl Data cho bài toán Text Classification - Phân loại sản phẩm

## Mục tiêu:
- Crawl 900 sản phẩm từ các sàn TMĐT (300 mỗi danh mục)
- 3 danh mục: **Điện thoại & Phụ kiện**, **Laptop & Máy tính**, **Thời trang Nam**
- Các cột: `product_name`, `describe`, `brand`, `category`

In [ ]:
# Cài đặt các thư viện cần thiết
%pip install requests pandas tqdm fake-useragent -q

In [2]:
import requests
import pandas as pd
import time
import random
import json
from tqdm import tqdm
from fake_useragent import UserAgent
import warnings
warnings.filterwarnings('ignore')

# Khởi tạo User Agent
ua = UserAgent()

print("✅ Import thành công!")

✅ Import thành công!


## 1. Định nghĩa các hàm crawl từ Tiki API

Tiki có API công khai cho phép lấy thông tin sản phẩm theo category.

In [3]:
# Cấu hình các category trên Tiki
TIKI_CATEGORIES = {
    'Điện thoại & Phụ kiện': {
        'category_id': 1789,  # Điện thoại - Máy tính bảng
        'sub_categories': [1795, 8060, 4535, 8062, 8063, 1801, 1802]  # Điện thoại, Phụ kiện, Tablet, etc.
    },
    'Laptop & Máy tính': {
        'category_id': 1846,  # Laptop - Máy vi tính - Linh kiện
        'sub_categories': [8095, 1847, 1882, 1883, 8099, 9202]  # Laptop, PC, Linh kiện, etc.
    },
    'Thời trang Nam': {
        'category_id': 915,  # Thời trang nam
        'sub_categories': [916, 918, 920, 921, 925, 927, 931]  # Áo, Quần, Giày, etc.
    }
}

def get_headers():
    """Tạo headers với random user agent"""
    return {
        'User-Agent': ua.random,
        'Accept': 'application/json',
        'Accept-Language': 'vi-VN,vi;q=0.9,en-US;q=0.8,en;q=0.7',
        'Referer': 'https://tiki.vn/',
        'x-guest-token': 'null',
    }

def crawl_tiki_products(category_id, limit=40, page=1):
    """
    Crawl sản phẩm từ Tiki API
    """
    url = "https://tiki.vn/api/personalish/v1/blocks/listings"
    
    params = {
        'limit': limit,
        'include': 'advertisement',
        'aggregations': 2,
        'version': 'home-persionalized',
        'trackity_id': '',
        'category': category_id,
        'page': page,
        'urlKey': '',
    }
    
    try:
        response = requests.get(url, headers=get_headers(), params=params, timeout=10)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Error: {response.status_code}")
            return None
    except Exception as e:
        print(f"Exception: {e}")
        return None

def get_product_detail(product_id):
    """
    Lấy chi tiết sản phẩm từ Tiki
    """
    url = f"https://tiki.vn/api/v2/products/{product_id}"
    
    params = {
        'platform': 'web',
        'spid': product_id,
        'version': 3
    }
    
    try:
        response = requests.get(url, headers=get_headers(), params=params, timeout=10)
        if response.status_code == 200:
            return response.json()
        return None
    except:
        return None

print("✅ Đã định nghĩa các hàm crawl Tiki")

✅ Đã định nghĩa các hàm crawl Tiki


In [4]:
def crawl_category_tiki(category_name, category_info, target_count=300):
    """
    Crawl sản phẩm theo danh mục từ Tiki
    """
    products = []
    seen_ids = set()
    
    print(f"\n{'='*60}")
    print(f"🛒 Đang crawl danh mục: {category_name}")
    print(f"{'='*60}")
    
    # Crawl từ category chính và các sub-categories
    all_categories = [category_info['category_id']] + category_info.get('sub_categories', [])
    
    for cat_id in all_categories:
        if len(products) >= target_count:
            break
            
        print(f"\n📁 Đang crawl category ID: {cat_id}")
        
        for page in range(1, 15):  # Tối đa 15 trang mỗi category
            if len(products) >= target_count:
                break
                
            data = crawl_tiki_products(cat_id, limit=40, page=page)
            
            if not data or 'data' not in data:
                print(f"  ⚠️ Không có data tại page {page}")
                break
                
            items = data.get('data', [])
            if not items:
                break
                
            for item in items:
                if len(products) >= target_count:
                    break
                    
                product_id = item.get('id')
                if product_id in seen_ids:
                    continue
                seen_ids.add(product_id)
                
                # Lấy thông tin cơ bản
                name = item.get('name', '')
                brand = item.get('brand_name', '') or ''
                
                # Lấy mô tả từ short_description hoặc description
                description = item.get('short_description', '') or ''
                
                # Nếu mô tả ngắn, thử lấy chi tiết
                if len(description) < 50:
                    detail = get_product_detail(product_id)
                    if detail:
                        description = detail.get('description', '') or detail.get('short_description', '') or ''
                        if not brand:
                            brand = detail.get('brand', {}).get('name', '') if detail.get('brand') else ''
                    time.sleep(0.3)  # Delay để tránh bị block
                
                # Chỉ thêm sản phẩm có đủ thông tin và tiếng Việt
                if name and len(name) > 5:
                    products.append({
                        'product_name': name.strip(),
                        'describe': description.strip() if description else name,
                        'brand': brand.strip() if brand else 'Không rõ',
                        'category': category_name
                    })
            
            print(f"  📦 Page {page}: Đã thu thập {len(products)}/{target_count} sản phẩm")
            time.sleep(random.uniform(0.5, 1.5))  # Random delay
    
    print(f"\n✅ Hoàn thành crawl {category_name}: {len(products)} sản phẩm")
    return products[:target_count]

print("✅ Đã định nghĩa hàm crawl theo danh mục")

✅ Đã định nghĩa hàm crawl theo danh mục


## 2. Crawl dữ liệu từ Tiki

In [5]:
# Crawl danh mục: Điện thoại & Phụ kiện
phone_products = crawl_category_tiki(
    'Điện thoại & Phụ kiện', 
    TIKI_CATEGORIES['Điện thoại & Phụ kiện'],
    target_count=300
)

print(f"\n📊 Tổng số sản phẩm Điện thoại & Phụ kiện: {len(phone_products)}")


🛒 Đang crawl danh mục: Điện thoại & Phụ kiện

📁 Đang crawl category ID: 1789
  📦 Page 1: Đã thu thập 40/300 sản phẩm
  📦 Page 2: Đã thu thập 80/300 sản phẩm
  📦 Page 3: Đã thu thập 120/300 sản phẩm
  📦 Page 4: Đã thu thập 160/300 sản phẩm
  📦 Page 5: Đã thu thập 200/300 sản phẩm
  📦 Page 6: Đã thu thập 220/300 sản phẩm

📁 Đang crawl category ID: 1795
  📦 Page 1: Đã thu thập 220/300 sản phẩm
  📦 Page 2: Đã thu thập 220/300 sản phẩm
  📦 Page 3: Đã thu thập 220/300 sản phẩm
  📦 Page 4: Đã thu thập 220/300 sản phẩm

📁 Đang crawl category ID: 8060
  📦 Page 1: Đã thu thập 260/300 sản phẩm
  📦 Page 2: Đã thu thập 300/300 sản phẩm

✅ Hoàn thành crawl Điện thoại & Phụ kiện: 300 sản phẩm

📊 Tổng số sản phẩm Điện thoại & Phụ kiện: 300


In [6]:
# Crawl danh mục: Laptop & Máy tính
laptop_products = crawl_category_tiki(
    'Laptop & Máy tính', 
    TIKI_CATEGORIES['Laptop & Máy tính'],
    target_count=300
)

print(f"\n📊 Tổng số sản phẩm Laptop & Máy tính: {len(laptop_products)}")


🛒 Đang crawl danh mục: Laptop & Máy tính

📁 Đang crawl category ID: 1846
  📦 Page 1: Đã thu thập 40/300 sản phẩm
  📦 Page 2: Đã thu thập 80/300 sản phẩm
  📦 Page 3: Đã thu thập 120/300 sản phẩm
  📦 Page 4: Đã thu thập 160/300 sản phẩm
  📦 Page 5: Đã thu thập 199/300 sản phẩm
  📦 Page 6: Đã thu thập 239/300 sản phẩm
  📦 Page 7: Đã thu thập 279/300 sản phẩm
  📦 Page 8: Đã thu thập 300/300 sản phẩm

✅ Hoàn thành crawl Laptop & Máy tính: 300 sản phẩm

📊 Tổng số sản phẩm Laptop & Máy tính: 300


In [7]:
# Crawl danh mục: Thời trang Nam
fashion_products = crawl_category_tiki(
    'Thời trang Nam', 
    TIKI_CATEGORIES['Thời trang Nam'],
    target_count=300
)

print(f"\n📊 Tổng số sản phẩm Thời trang Nam: {len(fashion_products)}")


🛒 Đang crawl danh mục: Thời trang Nam

📁 Đang crawl category ID: 915
  📦 Page 1: Đã thu thập 40/300 sản phẩm
  📦 Page 2: Đã thu thập 80/300 sản phẩm
  📦 Page 3: Đã thu thập 120/300 sản phẩm
  📦 Page 4: Đã thu thập 160/300 sản phẩm
  📦 Page 5: Đã thu thập 200/300 sản phẩm
  📦 Page 6: Đã thu thập 240/300 sản phẩm
  📦 Page 7: Đã thu thập 280/300 sản phẩm
  📦 Page 8: Đã thu thập 300/300 sản phẩm

✅ Hoàn thành crawl Thời trang Nam: 300 sản phẩm

📊 Tổng số sản phẩm Thời trang Nam: 300


## 3. Tổng hợp và làm sạch dữ liệu

In [8]:
# Tổng hợp tất cả dữ liệu
all_products = phone_products + laptop_products + fashion_products

# Tạo DataFrame
df = pd.DataFrame(all_products)

# Xem thông tin cơ bản
print("📊 THÔNG TIN DATASET")
print("="*60)
print(f"Tổng số sản phẩm: {len(df)}")
print(f"\n📁 Phân bố theo danh mục:")
print(df['category'].value_counts())
print(f"\n📋 Các cột trong dataset:")
print(df.columns.tolist())
print(f"\n🔍 Thông tin chi tiết:")
df.info()

📊 THÔNG TIN DATASET
Tổng số sản phẩm: 900

📁 Phân bố theo danh mục:
category
Điện thoại & Phụ kiện    300
Laptop & Máy tính        300
Thời trang Nam           300
Name: count, dtype: int64

📋 Các cột trong dataset:
['product_name', 'describe', 'brand', 'category']

🔍 Thông tin chi tiết:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   product_name  900 non-null    object
 1   describe      900 non-null    object
 2   brand         900 non-null    object
 3   category      900 non-null    object
dtypes: object(4)
memory usage: 28.3+ KB


In [9]:
# Xem mẫu dữ liệu
print("📝 MẪU DỮ LIỆU (5 sản phẩm đầu mỗi danh mục)")
print("="*80)

for category in df['category'].unique():
    print(f"\n🏷️ {category}:")
    print("-"*80)
    sample = df[df['category'] == category].head(3)
    for idx, row in sample.iterrows():
        print(f"📦 Tên: {row['product_name'][:80]}...")
        print(f"   Thương hiệu: {row['brand']}")
        print(f"   Mô tả: {row['describe'][:100]}..." if len(row['describe']) > 100 else f"   Mô tả: {row['describe']}")
        print()

📝 MẪU DỮ LIỆU (5 sản phẩm đầu mỗi danh mục)

🏷️ Điện thoại & Phụ kiện:
--------------------------------------------------------------------------------
📦 Tên: Máy Tính Bảng Galaxy Tab S11 Wifi (12GB/128GB) - Hàng Chính Hãng...
   Thương hiệu: Samsung
   Mô tả: <p><img src="https://salt.tikicdn.com/ts/tmp/50/33/61/6f12de442d85cc29ddd7c2f979d2e7f1.jpg" alt="" w...

📦 Tên: Điện thoại OPPO A3 8GB/128GB - Hàng Chính Hãng...
   Thương hiệu: OPPO
   Mô tả: <div class="ProductContent_description-container__miT3z">
<p><strong>Chiếc điện thoại bền bỉ, hiệu s...

📦 Tên: Điện thoại OPPO Reno13 F 5G 12GB/512GB - Hàng Chính Hãng...
   Thương hiệu: OPPO
   Mô tả: <p><strong>Điện thoại cho hiệu năng mượt mà và kết nối 5G trực tuyến nhanh hơn cùng chip Snapdragon ...


🏷️ Laptop & Máy tính:
--------------------------------------------------------------------------------
📦 Tên: Màn Hình KTC H27T13 2K 27 inch QHD 120Hz NEW 1ms - Hàng Chính Hãng...
   Thương hiệu: KTC
   Mô tả: <h2><strong>Màn hình KTC H2

In [10]:
import re
from html import unescape

def clean_text(text):
    """
    Làm sạch text: loại bỏ HTML tags, ký tự đặc biệt, khoảng trắng thừa
    """
    if not text or pd.isna(text):
        return ""
    
    # Decode HTML entities
    text = unescape(str(text))
    
    # Loại bỏ HTML tags
    text = re.sub(r'<[^>]+>', ' ', text)
    
    # Loại bỏ URLs
    text = re.sub(r'http[s]?://\S+', '', text)
    
    # Loại bỏ các ký tự đặc biệt không cần thiết nhưng giữ lại tiếng Việt
    text = re.sub(r'[^\w\sàáảãạăằắẳẵặâầấẩẫậèéẻẽẹêềếểễệìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵđÀÁẢÃẠĂẰẮẲẴẶÂẦẤẨẪẬÈÉẺẼẸÊỀẾỂỄỆÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴĐ,.\-/+%()]', ' ', text)
    
    # Loại bỏ khoảng trắng thừa
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# Làm sạch dữ liệu
df['product_name'] = df['product_name'].apply(clean_text)
df['describe'] = df['describe'].apply(clean_text)
df['brand'] = df['brand'].apply(clean_text)

# Loại bỏ các dòng có product_name hoặc describe rỗng
df = df[(df['product_name'].str.len() > 10) & (df['describe'].str.len() > 10)]

print("✅ Đã làm sạch dữ liệu")
print(f"📊 Số sản phẩm sau khi làm sạch: {len(df)}")

✅ Đã làm sạch dữ liệu
📊 Số sản phẩm sau khi làm sạch: 900


In [11]:
# Xem mẫu dữ liệu sau khi làm sạch
print("📝 MẪU DỮ LIỆU SAU KHI LÀM SẠCH")
print("="*80)

for category in df['category'].unique():
    print(f"\n🏷️ {category}:")
    print("-"*80)
    sample = df[df['category'] == category].head(2)
    for idx, row in sample.iterrows():
        print(f"📦 Tên: {row['product_name'][:100]}")
        print(f"   Thương hiệu: {row['brand']}")
        desc_preview = row['describe'][:150] + "..." if len(row['describe']) > 150 else row['describe']
        print(f"   Mô tả: {desc_preview}")
        print()

📝 MẪU DỮ LIỆU SAU KHI LÀM SẠCH

🏷️ Điện thoại & Phụ kiện:
--------------------------------------------------------------------------------
📦 Tên: Máy Tính Bảng Galaxy Tab S11 Wifi (12GB/128GB) - Hàng Chính Hãng
   Thương hiệu: Samsung
   Mô tả: Giá sản phẩm trên Tiki đã bao gồm thuế theo luật hiện hành. Bên cạnh đó, tuỳ vào loại sản phẩm, hình thức và địa chỉ giao hàng mà có thể phát sinh thê...

📦 Tên: Điện thoại OPPO A3 8GB/128GB - Hàng Chính Hãng
   Thương hiệu: OPPO
   Mô tả: Chiếc điện thoại bền bỉ, hiệu suất tốt bởi cấu hình từ Snapdragon 6s 4G Gen 1, pin siêu lớn 5.100mAh, sạc nhanh 45W, độ bền chuẩn quân đội và kháng bụ...


🏷️ Laptop & Máy tính:
--------------------------------------------------------------------------------
📦 Tên: Màn Hình KTC H27T13 2K 27 inch QHD 120Hz NEW 1ms - Hàng Chính Hãng
   Thương hiệu: KTC
   Mô tả: Màn hình KTC H27T13 Sự lựa chọn hoàn hảo cho không gian làm việc hiện đại Bạn đang tìm kiếm một màn hình vừa có thiết kế đẹp mắt, vừa có hiệu năng mạn..

In [12]:
# Thống kê đa dạng thương hiệu trong mỗi danh mục
print("📊 ĐA DẠNG THƯƠNG HIỆU TRONG MỖI DANH MỤC")
print("="*80)

for category in df['category'].unique():
    cat_df = df[df['category'] == category]
    brands = cat_df['brand'].value_counts()
    print(f"\n🏷️ {category}:")
    print(f"   Tổng số thương hiệu: {len(brands)}")
    print(f"   Top 10 thương hiệu phổ biến:")
    for brand, count in brands.head(10).items():
        print(f"      - {brand}: {count} sản phẩm")

📊 ĐA DẠNG THƯƠNG HIỆU TRONG MỖI DANH MỤC

🏷️ Điện thoại & Phụ kiện:
   Tổng số thương hiệu: 46
   Top 10 thương hiệu phổ biến:
      - Xiaomi: 54 sản phẩm
      - Samsung: 39 sản phẩm
      - Panasonic: 21 sản phẩm
      - OPPO: 20 sản phẩm
      - Orico: 17 sản phẩm
      - Realme: 16 sản phẩm
      - Kindle: 14 sản phẩm
      - Apple: 12 sản phẩm
      - SanDisk: 9 sản phẩm
      - WD: 8 sản phẩm

🏷️ Laptop & Máy tính:
   Tổng số thương hiệu: 96
   Top 10 thương hiệu phổ biến:
      - TP-Link: 38 sản phẩm
      - Alpha Cartridge: 23 sản phẩm
      - Brother: 19 sản phẩm
      - OEM: 11 sản phẩm
      - Orico: 10 sản phẩm
      - Viet Toner: 10 sản phẩm
      - UGREEN: 7 sản phẩm
      - MINHHA: 7 sản phẩm
      - MISUFUJl: 6 sản phẩm
      - NoBrand: 6 sản phẩm

🏷️ Thời trang Nam:
   Tổng số thương hiệu: 44
   Top 10 thương hiệu phổ biến:
      - OEM: 87 sản phẩm
      - ARADO FASHION: 20 sản phẩm
      - MRM Manlywear: 20 sản phẩm
      - SZone: 19 sản phẩm
      - Ares man: 15 sản 

## 4. Lưu dữ liệu

In [ ]:
# Reset index
df = df.reset_index(drop=True)

# Lưu file CSV
csv_path = 'product_classification_dataset.xlsx'
df.to_excel(csv_path, index=False, encoding='utf-8-sig')
print(f"Đã lưu file CSV: {csv_path}")

✅ Đã lưu file CSV: product_classification_dataset.csv
✅ Đã lưu file JSON: product_classification_dataset.json

📁 Kích thước file:
   - CSV: 2178.52 KB
   - JSON: 2250.17 KB


In [1]:
import pandas as pd